# Creating a system LLM_based-on Tutor
This is a technial tutor that use GPT-4o to answer questions in a step-by-step manner. It is based on the Tutor LLM model.

In [5]:
# Import libraries
import os
import json
import string
import requests
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import display, Markdown, update_display
from openai import OpenAI

import datetime
print(f"This notebook was last run on {datetime.datetime.now():%Y-%m-%d %H:%M:%S}")

This notebook was last run on 2025-09-17 17:07:06


## Configure the environment parameters

In [6]:
MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = "llama3.2"

In [9]:
# Load model
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("The API Key is in right format!")
else:
    print("There's a problem in API key!!!")
    
openao = OpenAI()

headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

The API Key is in right format!


In [ ]:
# here is the question; type over this to ask something new
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [11]:
def messages_for(question, system_prompt):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

In [22]:
client = OpenAI()
def stream_answer(system_prompt, user_prompt):
    stream = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""   # None thì thay bằng ""
        response += str(delta)
        cleaned = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(cleaned), display_id=display_handle.display_id)


In [23]:
system_prompt = "You are a helpful Python teacher. Especially, you have a nice mindset in coding skills."
# user_prompt = """
# Please explain what this code does and why:
# yield from {book.get("author") for book in books if book.get("author")}
# """

user_prompt = """
Explain the core principles of functional programming in a simple and concise way. 
Cover concepts such as immutability, pure functions, higher-order functions, and recursion. 
Use short code examples in Python to illustrate.
"""


In [20]:
# Streaming response from gpt-4o-mini
client = OpenAI()
with client.chat.completions.stream(
    model="gpt-4o-mini",
    messages=messages_for(system_prompt, user_prompt),
) as stream:
    for event in stream:
        if event.type == "message.delta":
            # Print new text as it streams in
            print(event.delta, end="", flush=True)
        elif event.type == "message.completed":
            print("\n--- done ---")

In [24]:
final_answer = stream_answer(system_prompt, user_prompt)

Functional programming is a programming paradigm that treats computation as the evaluation of mathematical functions and avoids changing state and mutable data. Here are the core principles of functional programming explained simply:

### 1. Immutability
In functional programming, data is immutable, meaning once it's created, it cannot be changed. This helps to avoid side effects and makes programs easier to reason about.

**Example:**
python
# Instead of modifying a list, create a new one
original_list = [1, 2, 3]
new_list = original_list + [4]  # original_list remains unchanged
print(original_list)  # Output: [1, 2, 3]
print(new_list)       # Output: [1, 2, 3, 4]


### 2. Pure Functions
A pure function is a function where the output value is determined only by its input values, with no side effects (no external state changes).

**Example:**
python
def add(x, y):
    return x + y  # Same input will always yield the same output

print(add(2, 3))  # Output: 5
print(add(2, 3))  # Output: 5 (no side effects)


### 3. Higher-Order Functions
Higher-order functions are functions that can take other functions as arguments, return them as results, or both. They enable a powerful way to abstract behavior.

**Example:**
python
def apply_function(func, value):
    return func(value)

def square(x):
    return x * x

result = apply_function(square, 4)
print(result)  # Output: 16


### 4. Recursion
Recursion is a technique where a function calls itself to solve a smaller instance of the same problem. It's commonly used as an alternative to loops in functional programming.

**Example:**
python
def factorial(n):
    if n == 0:  # Base case
        return 1
    else:
        return n * factorial(n - 1)  # Recursive step

print(factorial(5))  # Output: 120


### Summary
Functional programming emphasizes immutability, pure functions, higher-order functions, and recursion. This approach can lead to cleaner, more predictable code, making it easier to debug and reason about.